In [0]:
import pandas as pd
import numpy as np


In [0]:
column_names = ['JokeCount'] + [f'Joke{num + 1}' for num in range(100)]
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',header=None, names=column_names)

In [21]:
dataframe = df.drop(columns=['JokeCount'])
x= dataframe.values
print(x.shape)

(24983, 100)


In [0]:
def replace(original, percentage=0.1):
  new_data = original.copy()
  rated = np.where(x!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])



In [22]:
new_x, idx = replace(x, 0.1)
rated = np.where(x!=99)
new_rated = np.where(new_x!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [42]:
n_latent_factors = 2

user_ratings = new_x
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(user_ratings)

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [0]:
def predict_rating(user_id, item_id):
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]

    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values

    
    return err
    

def sgd(iterations=100):
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [50]:
sgd()

22.87280286377694
18.46733620306312
17.495675919723727
17.253735905753164
16.987850925689646
16.573928111109836
16.04307138055244
15.538251794300002
15.164440282204753
14.923098151941703


In [54]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-2.95295612, -2.51214893, -6.0108158 , ..., -2.72743743,
        -7.63098737, -3.70262349],
       [ 1.33121429,  1.91520732,  3.02617371, ...,  4.06466473,
         4.34267315,  4.45531163],
       [ 6.87602977,  4.65271837,  5.87011237, ...,  7.17741111,
         8.41495533,  7.15567856],
       ...,
       [-1.08701616, -0.17005071, -2.25530525, ..., -0.76380538,
        -3.48715017, -1.23261243],
       [-1.96823151, -2.58650354, -4.46812035, ..., -0.89928965,
        -3.64783866, -2.03845329],
       [ 2.30392425,  1.23710248,  1.06049164, ...,  5.40615784,
         3.76658936,  4.80460741]])

In [55]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(-7.82, -2.952956122557951)","(8.79, -2.5121489274449114)","(-9.66, -6.010815803036255)","(-8.16, -9.145483325058729)","(-7.52, -2.892192553084946)","(-8.5, -3.1917252504317983)","(-9.85, -2.667870658081202)","(4.17, -2.950864054464721)","(-8.98, -8.580786837662234)","(-4.76, -3.5865078661555483)","(-8.5, -2.5757888336535584)","(-6.75, -1.9618283317578824)","(-7.18, -4.808338905381398)","(8.45, -1.2084815298397722)","(-7.18, -4.659749559706356)","(-7.52, -9.039611962014316)","(-7.43, -3.9865043375314526)","(-9.81, -3.935903156624497)","(-9.85, -3.6283983682089858)","(-9.85, -6.255362628566881)","(-9.37, 0.7309000732345222)","(1.5, -3.5939799101104684)","(-4.37, -6.603006603410117)","(-9.81, -10.758100546956864)","(-8.5, -4.284873389806812)","(1.12, -1.0613501925510842)","(7.82, 2.937578351843467)","(2.86, -2.564232750027569)","(9.13, 5.563876236988658)","(-7.43, -9.258973957099004)","(2.14, 0.6670136144285552)","(-4.08, 3.75514926478975)","(-9.08, -9.448620358654189)","(7.82, -2.776909268000762)","(5.05, 3.83117687409634)","(4.95, 4.031604493400986)","(-9.17, -7.750490803300154)","(-8.4, -3.3869626708932086)","(-8.4, -2.8851640459905554)","(-8.4, -3.848413299687155)",...,"(8.59, 0.5591642430741479)","(3.59, 1.827556310813174)","(-6.84, -5.570345206255686)","(-9.03, -7.603022695114808)","(2.82, 1.8568212095201644)","(-1.36, 1.153175896024951)","(-9.08, -8.902146272758987)","(8.3, 1.385151148013527)","(5.68, 2.066345274741019)","(-4.81, -5.832023342339336)","(99.0, -9.00246076486741)","(99.0, -0.5504534458427833)","(99.0, -6.835062339972257)","(99.0, -11.169258755841204)","(99.0, -8.079766400768161)","(99.0, -1.3588288950894996)","(99.0, -7.338167971992393)","(-9.42, -5.3382750553422555)","(99.0, -8.711547867315195)","(99.0, -5.439264289298094)","(99.0, -2.2927808592096053)","(-7.72, -6.915843744743707)","(99.0, -1.7800411244179903)","(99.0, -6.969949844160801)","(99.0, -6.135951902633001)","(99.0, -7.814706836153903)","(99.0, -2.623613052123238)","(99.0, -2.780258866981563)","(99.0, 1.8070024364347554)","(99.0, -7.41092005518999)","(2.82, -2.625501096031326)","(99.0, -4.869328691850638)","(99.0, -1.037697768762868)","(99.0, -5.976404379821736)","(99.0, -4.9409820575364245)","(99.0, -4.155478875589974)","(-5.63, -2.8252255423489667)","(99.0, -2.7274374331381517)","(99.0, -7.63098737184424)","(99.0, -3.702623488199062)"
1,"(4.08, 1.3312142907790632)","(-0.29, 1.9152073238166216)","(6.36, 3.026173707127834)","(4.37, 5.522922079214751)","(-2.38, 3.0558274713858955)","(-9.66, 3.141032390226728)","(-0.73, 1.5929239598189757)","(-5.34, 0.7419791551447024)","(8.88, 3.5892702754944876)","(9.22, 1.514527426193227)","(6.75, 2.404313399096912)","(8.64, 2.740572472991548)","(4.42, 2.199375669071285)","(7.43, 2.6888707641172642)","(4.56, 1.2109029915181586)","(-0.97, 3.3459671223767984)","(4.66, 3.0140195845081763)","(-0.68, 1.4236229158544413)","(3.3, 2.5558074833829183)","(-1.21, 3.664691465137268)","(0.87, 2.41115708575082)","(8.64, 3.0001091200322225)","(8.35, 2.938934627563397)","(9.17, 4.942432376600052)","(0.05, 3.252448353370875)","(7.57, 2.7444913060829674)","(4.71, 0.7130462507760702)","(0.87, 2.0382388258856574)","(-0.39, -0.7209453058674771)","(6.99, 5.029739020386564)","(6.5, 1.9598028790744935)","(-0.92, -0.20907017581827503)","(7.14, 4.648086016559668)","(9.03, 3.327131494965611)","(-1.8, 0.014276718093483534)","(0.73, -0.36546600974093485)","(7.09, 2.974642240143164)","(3.4, 1.744310258999557)","(-0.87, 3.5509322601589917)","(7.91, 3.0062740382128608)",...,"(-6.7, 1.5694661870709161)","(-3.35, 0.2925576578142897)","(-9.03, 2.71598382687427)","(4.47, 3.126353719066162)","(4.08, 1.7588439366960829)","(-3.83, 0.9272397843597412)","(8.74, 4.255154376183591)","(1.12, 0.4938641716187921)","(0.78, 0

In [56]:
n_latent_factors = 5
user_ratings = x
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))
sgd()
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions


25.572412467358788
24.015454600322997
21.39314615950442
17.81162634597321
17.39698617908974
17.1684014172776
16.861629200504762
16.452975049051457
15.999242510461405
15.577278794021607


array([[-3.46926959, -3.32218102, -5.79805469, ..., -4.05991754,
        -6.38561542, -4.56387645],
       [ 3.04593844,  3.25011825,  4.32052918, ...,  4.02836763,
         3.1092554 ,  4.83612983],
       [ 5.83120977,  6.1628579 ,  6.61422971, ...,  7.50745989,
         7.28922745,  8.24634464],
       ...,
       [-2.30056581, -0.96090207, -2.13496708, ...,  0.16165307,
        -2.16110752,  0.12595714],
       [-1.96242963, -0.09812465, -2.41686278, ..., -0.0190166 ,
        -5.45351865,  0.46306429],
       [ 3.3359256 ,  2.54697426,  2.83968273, ...,  3.19770867,
         2.19336291,  3.76213506]])

In [57]:

values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data= pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(-7.82, -3.469269585136326)","(8.79, -3.322181022923023)","(-9.66, -5.798054693192957)","(-8.16, -10.575595563387377)","(-7.52, -2.0631382535544986)","(-8.5, -2.5103248769402895)","(-9.85, -1.7636401352214723)","(4.17, -2.1518374903204243)","(-8.98, -9.716839889083598)","(-4.76, -3.793074845019464)","(-8.5, -1.630826144152904)","(-6.75, -2.0685082255499787)","(-7.18, -5.821849677340179)","(8.45, -1.8760898363849359)","(-7.18, -4.716297313236048)","(-7.52, -8.883435816464829)","(-7.43, -4.207933685195258)","(-9.81, -3.52882478403436)","(-9.85, -2.236403061688237)","(-9.85, -4.420720293126135)","(-9.37, -0.3808901462709809)","(1.5, -4.868315820232805)","(-4.37, -5.549474558206362)","(-9.81, -11.674103470627179)","(-8.5, -4.803214377107404)","(1.12, -2.250195798937804)","(7.82, 3.138592373043512)","(2.86, -0.3631991796238742)","(9.13, 4.516204769923487)","(-7.43, -8.457324587449662)","(2.14, 0.746945908441691)","(-4.08, 4.350936684395373)","(-9.08, -9.256447628203846)","(7.82, -3.714243532404439)","(5.05, 3.447190870126565)","(4.95, 4.209173789396753)","(-9.17, -8.05154451330798)","(-8.4, -2.7265923914671246)","(-8.4, -3.0125440429826926)","(-8.4, -3.1424880499782653)",...,"(8.59, 0.971342433123894)","(3.59, 2.898806723321637)","(-6.84, -3.936207387834169)","(-9.03, -7.609408967437118)","(2.82, 0.22299881626478274)","(-1.36, 1.7499556173608095)","(-9.08, -10.44033413167448)","(8.3, 2.424238870517705)","(5.68, 1.94911305386618)","(-4.81, -5.634532636815999)","(99.0, -6.360695355992182)","(99.0, 0.09382671261054232)","(99.0, -5.536639627246737)","(99.0, -10.18801178086206)","(99.0, -7.813625640553136)","(99.0, -0.7038561490403901)","(99.0, -5.788679276265358)","(-9.42, -3.7009233144759444)","(99.0, -7.323384609697967)","(99.0, -4.9912716274971505)","(99.0, -3.0796184916604736)","(-7.72, -4.8547203739435885)","(99.0, -1.9460604468344478)","(99.0, -4.804875417813644)","(99.0, -6.072620548517767)","(99.0, -7.0990620614674045)","(99.0, -2.3702125258924536)","(99.0, -1.9808378148815682)","(99.0, 2.9126651113699804)","(99.0, -5.066006926889475)","(2.82, -2.40429882523567)","(99.0, -4.084037897142856)","(99.0, -0.12426541629872234)","(99.0, -5.69649604828041)","(99.0, -4.381062551556927)","(99.0, -3.554662700782127)","(-5.63, -2.3088645023767636)","(99.0, -4.059917540588647)","(99.0, -6.385615422723956)","(99.0, -4.563876450400107)"
1,"(4.08, 3.045938440308413)","(-0.29, 3.2501182490750558)","(6.36, 4.320529179388985)","(4.37, 6.23825871846785)","(-2.38, 1.7402272974700979)","(-9.66, 2.881917578681667)","(-0.73, 0.2781815624241445)","(-5.34, 0.412919975892439)","(8.88, 5.491119723380994)","(9.22, 3.4065981956068176)","(6.75, 2.7633479241883774)","(8.64, 3.084660962067857)","(4.42, 2.7256548536863914)","(7.43, 3.1593428687777547)","(4.56, 1.3377996280636262)","(-0.97, 2.6431408014717395)","(4.66, 2.3346446924234074)","(-0.68, 0.8656189962653571)","(3.3, 0.53767133178273)","(-1.21, 1.048993340542733)","(0.87, 3.009731511837161)","(8.64, 4.57011766815352)","(8.35, 3.2278455285383942)","(9.17, 6.223676273469503)","(0.05, 4.155361280894386)","(7.57, 3.491760853961384)","(4.71, 0.8038875380579128)","(0.87, 0.49131137647090484)","(-0.39, -0.0010640955983064614)","(6.99, 4.774131723343428)","(6.5, 2.0501910709675126)","(-0.92, -0.7847950976032931)","(7.14, 4.997528128512348)","(9.03, 4.103454803371736)","(-1.8, 0.3035029628573843)","(0.73, -0.11274833095443844)","(7.09, 3.7962866814943403)","(3.4, 2.4071860456622884)","(-0.87, 3.4506727874680507)","(7.91, 2.91244734498305)",...,"(-6.7, 1.4074753922975813)","(-3.35, -0.29653796733788235)","(-9.03, 1.6391058125409836)","(4.47, 4.015471388078169)","(4.08, 2.720227376092727)","(-3.83, 0.7247527143649783)","(8.74, 6.108890870040165)","(1.12, -0.27399449065850595)","(0